In [2]:
import pandas as pd
import numpy as np
from functools import reduce
import random
from tqdm import tqdm
from scipy.sparse import csr_matrix, coo_matrix
import time
import pickle

In [3]:
df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')
years = pd.read_csv('~/input/data/train/years.tsv', sep = '\t')
directors = pd.read_csv('~/input/data/train/directors.tsv', sep = '\t')
genres = pd.read_csv('~/input/data/train/genres.tsv', sep = '\t')
titles = pd.read_csv('~/input/data/train/titles.tsv', sep = '\t')
writers = pd.read_csv('~/input/data/train/writers.tsv', sep = '\t')
dfs = [df,years,directors,genres,writers]

# Negative Sampling

In [3]:
interaction = df[['user','item']].values

In [4]:
interaction[:10]

array([[  11, 4643],
       [  11,  170],
       [  11,  531],
       [  11,  616],
       [  11, 2140],
       [  11, 2722],
       [  11, 2313],
       [  11, 2688],
       [  11, 2428],
       [  11, 3113]])

In [5]:
len(interaction)

5154471

In [6]:
users = interaction[:,0]
items = interaction[:,1]

In [8]:
interaction_mtx = csr_matrix(([1]*len(interaction), (users, items)), shape = (df['user'].max()+1, df['item'].max()+1))

In [9]:
interaction_mtx.shape # 0-th row & 0-th is 0

(138494, 119146)

In [12]:
type(interaction_mtx[11,1])

numpy.int64

In [18]:
user_idx = df.user.astype('category').cat.codes.values
item_idx = df.item.astype('category').cat.codes.values
df['interaction'] = 1

In [24]:
sparse_mtx = csr_matrix((df.interaction, (df.user, df.item)))

In [52]:
tmp = np.argwhere(sparse_mtx[11]==0)
tmp[:,0] = 11
tmp

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


array([[    11,      0],
       [    11,      2],
       [    11,      3],
       ...,
       [    11, 119143],
       [    11, 119144],
       [    11, 119145]], dtype=int32)

In [53]:
# neg_samples = []
# for i,row in enumerate(sparse_mtx) : 
#     # start = time.time()
#     n_non_0 = len(np.nonzero(row)[0])
#     zero_indices = np.argwhere(row==0)
#     zero_indices[:,0] = i
#     if n_non_0 > len(zero_indices) : 
#         neg_indices = zero_indices.tolist()
#     else : 
#         neg_indices = random.sample(zero_indices.tolist(), n_non_0)
        
#     neg_samples.extend(neg_indices)
#     # end = time.time()
#     # if (i!=0)&(i%10000 == 0) : 
#     #     print("time : ", end - start)

0it [00:00, ?it/s]/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
10001it [39:50,  3.57it/s]

time :  0.3899500370025635


12628it [53:07,  3.96it/s]


KeyboardInterrupt: 

In [7]:
merge_df.head()

,user,item,time,year,director,genre,writer
0,11,4643,1230782529,2001.0,nm0000318,Action,nm0099541
1,11,4643,1230782529,2001.0,nm0000318,Action,nm0115310
2,11,4643,1230782529,2001.0,nm0000318,Action,nm0465199
3,11,4643,1230782529,2001.0,nm0000318,Action,nm0742797
4,11,4643,1230782529,2001.0,nm0000318,Adventure,nm0099541


In [8]:
merge_idx_df.head() # global time -> local time

,user_idx,item_idx,time_idx,year_idx,director_idx,genre_idx,writer_idx
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,2
3,0,0,0,0,0,0,3
4,0,0,0,0,0,1,0


In [19]:
merge_idx_df.nunique().tolist() # n_users, n_items, n_time, n_year, n_director, n_genre, n_writer

[31360, 6807, 5074973, 94, 1341, 18, 2990]

In [41]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
class FMDataset(Dataset):
    def __init__(self, data) : 
        self.data = data
        self.target = data['item_idx'] # item_idx
        self.input_data = data[[col for col in data.columns if col not in self.target.name]] # merge_idx_df
    
    
    def __getitem__(self, index) : 
        return self.input_data[index], self.target[index]
        
    def __len__(self, target) : 
        return len(self.target)
    
    def negative_sampling(self) :
        self.data.groupby(['user_idx'])

In [57]:
# with open('/opt/ml/bk/data/neg_sample.pkl', 'wb') as f:
#     pickle.dump(neg_samples, f)

In [3]:
with open('/opt/ml/bk/data/neg_sample.pkl', 'rb') as f:
    neg_samples = pickle.load(f)

In [4]:
neg_samples = np.array(neg_samples)

In [5]:
neg_samples.shape

(5154471, 2)

In [6]:
df.shape

(5154471, 3)

In [7]:
neg_users = neg_samples[:,0]
neg_items = neg_samples[:,1]

In [8]:
neg_df = pd.DataFrame({'user':neg_users, 'item':neg_items, 'interaction':0})

In [9]:
neg_df

,user,item,interaction
0,11,60708,0
1,11,97674,0
2,11,76937,0
3,11,73183,0
4,11,30188,0
...,...,...,...
5154466,138493,10513,0
5154467,138493,116461,0
5154468,138493,13495,0
5154469,138493,32708,0


In [10]:
df_copy = df.copy()

In [11]:
df_copy['interaction'] = 1

In [12]:
neg_n_pos = pd.concat([df_copy, neg_df], axis = 0)

In [13]:
neg_df[neg_df.columns.difference(['interaction'])]

,item,user
0,60708,11
1,97674,11
2,76937,11
3,73183,11
4,30188,11
...,...,...
5154466,10513,138493
5154467,116461,138493
5154468,13495,138493
5154469,32708,138493


In [14]:
dfs[0] =  neg_n_pos

In [4]:
item_df = reduce(lambda left, right:pd.merge(left, right, on = 'item', how = 'left'), dfs[1:])

In [24]:
item_df.head()

,item,year,director,genre,writer
0,1348,1922,nm0003638,Horror,nm0831290
1,44587,1922,NaN,Documentary,NaN
2,44587,1922,NaN,Drama,NaN
3,4768,1922,nm0000485,Crime,nm0000485
4,4768,1922,nm0000485,Crime,nm0415167


In [27]:
item_df.set_index('item').sort_index().loc[1]

,year,director,genre,writer
item,,,,
1,1995,nm0005124,Comedy,nm0923736
1,1995,nm0005124,Animation,nm0923736
1,1995,nm0005124,Animation,nm0812513
1,1995,nm0005124,Animation,nm0710020
1,1995,nm0005124,Animation,nm0230032
1,1995,nm0005124,Animation,nm0169505
1,1995,nm0005124,Animation,nm0005124
1,1995,nm0005124,Animation,nm0004056
1,1995,nm0005124,Adventure,nm0923736


In [3]:
merge_df = reduce(lambda left, right:pd.merge(left, right, on='item', how='left'), dfs)

In [4]:
merge_df.columns

Index(['user', 'item', 'time', 'year', 'director', 'genre', 'writer'], dtype='object')

In [41]:
item_list = df['item'].unique().tolist()
user_list = df['user'].unique().tolist()
interaction_dict = dict(df.groupby('user')['item'].apply(list))

In [5]:
#
with open('/opt/ml/bk/data/no_inter_items.pkl', 'rb') as f:
    no_inter_dict = pickle.load(f)
    
no_inter_df = pd.DataFrame({'user':no_inter_dict.keys(), 'item':no_inter_dict.values()}).explode('item').reset_index(drop = True)

In [42]:
no_inter_dict = {}
for user in user_list :
    no_inter = list(set(item_list) - set(interaction_dict[user]))
    no_inter_dict[user] = no_inter

In [44]:
len(no_inter_dict)

31360

In [53]:
no_inter_df = pd.DataFrame({'user':no_inter_dict.keys(), 'item':no_inter_dict.values()}).explode('item')

In [57]:
no_inter_df = no_inter_df.reset_index(drop = True)

In [61]:
no_inter_df=no_inter_df.iloc[:,1:]

In [6]:
no_inter_df = no_inter_df.astype('int32')

In [10]:
len(no_inter_df)

208313049

In [7]:
no_inter_df.memory_usage()

Index          128
user     833252196
item     833252196
dtype: int64

In [66]:
no_inter_df.memory_usage()

Index           128
user     1666504392
item     1666504392
dtype: int64

In [8]:
pd.merge(no_inter_df, item_df, on='item', how = 'left') # memory error -> dataframe chunk

MemoryError: Unable to allocate 25.3 GiB for an array with shape (3, 1132193877) and data type object

In [45]:
# import pickle

# with open('/opt/ml/bk/data/no_inter_items.pkl', 'wb') as f:
#     pickle.dump(no_inter_dict, f)

In [12]:
user_groupby=merge_df.groupby('user')

In [16]:
next(iter(user_groupby))

(11,
       user  item        time    year   director      genre     writer
 0       11  4643  1230782529  2001.0  nm0000318     Action  nm0099541
 1       11  4643  1230782529  2001.0  nm0000318     Action  nm0115310
 2       11  4643  1230782529  2001.0  nm0000318     Action  nm0465199
 3       11  4643  1230782529  2001.0  nm0000318     Action  nm0742797
 4       11  4643  1230782529  2001.0  nm0000318  Adventure  nm0099541
 ...    ...   ...         ...     ...        ...        ...        ...
 4604    11  7153  1294796132  2003.0  nm0001392    Fantasy  nm0909638
 4605    11  4226  1294796159  2000.0  nm0634240    Mystery  nm0634240
 4606    11  4226  1294796159  2000.0  nm0634240    Mystery  nm0634300
 4607    11  4226  1294796159  2000.0  nm0634240   Thriller  nm0634240
 4608    11  4226  1294796159  2000.0  nm0634240   Thriller  nm0634300
 
 [4609 rows x 7 columns])

In [10]:
i=0
for user, group in merge_df.groupby('user') : 
    if i >=2 : 
        break
    print(group)
    i+=1

(11,       user  item        time    year   director      genre     writer
0       11  4643  1230782529  2001.0  nm0000318     Action  nm0099541
1       11  4643  1230782529  2001.0  nm0000318     Action  nm0115310
2       11  4643  1230782529  2001.0  nm0000318     Action  nm0465199
3       11  4643  1230782529  2001.0  nm0000318     Action  nm0742797
4       11  4643  1230782529  2001.0  nm0000318  Adventure  nm0099541
...    ...   ...         ...     ...        ...        ...        ...
4604    11  7153  1294796132  2003.0  nm0001392    Fantasy  nm0909638
4605    11  4226  1294796159  2000.0  nm0634240    Mystery  nm0634240
4606    11  4226  1294796159  2000.0  nm0634240    Mystery  nm0634300
4607    11  4226  1294796159  2000.0  nm0634240   Thriller  nm0634240
4608    11  4226  1294796159  2000.0  nm0634240   Thriller  nm0634300

[4609 rows x 7 columns])
(14,       user  item        time    year   director      genre     writer
4609    14  8961  1225308746  2004.0  nm0083348     Ac

In [7]:
merge_df.isnull().sum()

user              0
item              0
time              0
year           6920
director    1184231
genre             0
writer       695889
dtype: int64

In [18]:
merge_idx_df = pd.DataFrame()
merge_idx_df['user_idx'] = pd.factorize(merge_df['user'])[0]
merge_idx_df['item_idx'] = pd.factorize(merge_df['item'])[0]
merge_idx_df['time_idx'] = pd.factorize(merge_df['time'])[0]
merge_idx_df['year_idx'] = pd.factorize(merge_df['year'])[0]
merge_idx_df['director_idx'] = pd.factorize(merge_df['director'])[0]
merge_idx_df['genre_idx'] = pd.factorize(merge_df['genre'])[0]
merge_idx_df['writer_idx'] = pd.factorize(merge_df['writer'])[0]


In [24]:
field_dims = np.max(merge_idx_df)

In [26]:
field_dims

user_idx          31359
item_idx         119145
time_idx        5074972
year_idx             92
director_idx       1339
genre_idx            17
writer_idx         2988
dtype: int64

In [27]:
sum(field_dims)

5229912

In [31]:
import torch

In [37]:
torch.sum(torch.tensor(np.array((0, *np.cumsum(field_dims)[:-1]+1), dtype = np.int32)).unsqueeze(0), dim = 1)

tensor([21086744])

In [6]:
user_idx_dict = dict(zip(np.unique(pd.factorize(merge_df['user'])[0]), merge_df['user'].unique()))
item_idx_dict = dict(zip(np.unique(pd.factorize(merge_df['item'])[0]), merge_df['item'].unique()))
time_idx_dict = dict(zip(np.unique(pd.factorize(merge_df['time'])[0]), merge_df['time'].unique()))
year_idx_dict = dict(zip(np.unique(pd.factorize(merge_df['year'])[0]), merge_df['year'].unique()))
director_idx_dict = dict(zip(np.unique(pd.factorize(merge_df['director'])[0]), merge_df['director'].unique()))
genre_idx_dict = dict(zip(np.unique(pd.factorize(merge_df['genre'])[0]), merge_df['genre'].unique()))
writer_idx_dict = dict(zip(np.unique(pd.factorize(merge_df['writer'])[0]), merge_df['writer'].unique()))